### smet文件转换为csv文件

In [3]:
import os
import pandas as pd

input_folder = '/home/develop/Station/data/origin'
output_folder = '/home/develop/Station/data/csvwithHeader/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

smet_files = [f for f in os.listdir(input_folder) if f.endswith('.smet')]

for smet_file in smet_files:
    smet_file_path = os.path.join(input_folder, smet_file)
    data = pd.read_csv(smet_file_path, sep='\t', header=None)
    output_csv_path = os.path.join(output_folder, smet_file.replace('.smet', '.csv'))
    data.to_csv(output_csv_path, index=False)
    

### 去除头部Header

In [1]:
import os

input_dir = '/home/develop/Station/data/csvwithHeader/'
output_dir = '/home/develop/Station/data/csv/'

os.makedirs(output_dir, exist_ok=True)
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_filepath = os.path.join(input_dir, filename)     
        with open(input_filepath, 'r') as file:
            lines = file.readlines()
        start_index = None
        for i, line in enumerate(lines):
            if '[DATA]' in line:
                start_index = i + 1  
                break

        
        if start_index is not None:
            data_to_keep = lines[start_index:]
            output_filepath = os.path.join(output_dir, filename)
            with open(output_filepath, 'w') as output_file:
                output_file.writelines(data_to_keep)
            print(f"处理并保存文件: {filename}")
        else:
            print(f"文件 {filename} 中没有找到 [DATA] 标签")


处理并保存文件: station.csv


### 不同列之间用逗号隔开同时将内容转换为数值类型

In [2]:
import os
import pandas as pd

# 指定CSV文件夹路径
folder_path = '/home/develop/Station/data/csv/'

# 遍历文件夹中的所有CSV文件
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        data = pd.read_csv(file_path, delim_whitespace=True, header=None)
        
        # 假设时间戳是第一列（即索引0），我们将其分离出来
        timestamps = data.iloc[:, 0]
        
        # 对剩余的列进行数值转换，无法转换的内容置为NaN
        numeric_data = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
        
        # 将时间戳和数值数据重新合并
        processed_data = pd.concat([timestamps, numeric_data], axis=1)
        
        # 将DataFrame保存为新的CSV文件，使用逗号作为分隔符
        processed_data.to_csv(file_path, index=False, header=False, sep=',')
        
        print(f"文件 {filename} 已成功转换，并保留了时间戳列。")

文件 station.csv 已成功转换，并保留了时间戳列。


/tmp/ipykernel_340351/2389777297.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, header=None)


### 添加列名

In [3]:
import os
import pandas as pd

# 定义文件夹路径
folder_path = '/home/develop/Station/data/csv/'

# 定义表头（列名）
header = [
    "timestamp","sensible_heat", "latent_heat", "ground_heat", "ground_temperature", 
    "ground_heat_at_soil_interface", "rain_energy", "outgoing_long_wave_radiation", 
    "incoming_long_wave_radiation", "net_long_wave_radiation", "reflected_short_wave_radiation", 
    "incoming_short_wave_radiation", "net_short_wave_radiation", "parametrized_albedo", 
    "measured_albedo", "incoming_short_wave_on_horizontal", "direct_incoming_short_wave", 
    "diffuse_incoming_short_wave", "air_temperature", "surface_temperature(mod)", 
    "surface_temperature(meas)", "bottom_temperature", "relative_humidity", "wind_velocity", 
    "wind_velocity_drift", "wind_direction", "solid_precipitation_rate", "snow_height(mod)", 
    "snow_height(meas)", "hoar_size", "24h_wind_drift", "24h_height_of_new_snow", 
    "3d_sum_of_daily_height_of_new_snow", "snow_water_equivalent", "total_amount_of_water", 
    "erosion_mass_loss", "rain_rate", "virtual_lysimeter", 
    "virtual_lysimeter_under_the_soil", "sublimation_mass", "evaporated_mass", 
    "temperature@0.25m", "temperature@0.5m", "temperature@1m", "temperature@-0.25m", 
    "temperature@-0.1m", "profile_type", "stability_class", "z_Sdef", 
    "deformation_rate_stability_index", "z_Sn38", "natural_stability_index", "z_Sk38", 
    "Sk38_skier_stability_index", "z_SSI", "structural_stability_index", "z_S5", 
    "stability_index_5"
]


# 遍历文件夹中的所有 CSV 文件
for file_name in os.listdir(folder_path):    
    # 拼接完整路径
    file_path = os.path.join(folder_path, file_name)
    
    # 检查文件是否是 CSV 文件
    if not file_path.endswith('.csv'):
        continue
    
    try:
        # 读取 CSV 文件，以逗号为分隔符
        data = pd.read_csv(file_path, header=None, sep=',')
        
        # 检查列数是否与表头匹配
        if len(data.columns) == len(header):
            print(f"列数匹配，表头将被添加到文件中: {file_name}")
            # 添加表头
            data.columns = header

            # 将修改后的数据保存回 CSV 文件
            data.to_csv(file_path, index=False, sep=',')
            print(f"表头已成功添加到 {file_name}。")
        else:
            print(f"列数不匹配！文件: {file_name}，实际列数为 {len(data.columns)}，表头参数个数为 {len(header)}。")
    except Exception as e:
        print(f"处理文件 {file_name} 时出错: {e}")


列数匹配，表头将被添加到文件中: station.csv
表头已成功添加到 station.csv。


### 遍历文件，找到值不变的列

In [1]:
import pandas as pd
import os

# 定义文件路径
file_path = '/home/develop/Station/data/csv/station.csv'

# 读取 CSV 文件
data = pd.read_csv(file_path, header=0, sep=',', low_memory=False)

# 遍历每一列
for col in data.columns:
    # 检查列中是否所有值都相同
    unique_values = data[col].unique()
    if len(unique_values) == 1:
        # 输出列名和该列唯一的数值
        print(f"列 '{col}' 的所有值相同，值为: {unique_values[0]}")


列 'ground_heat' 的所有值相同，值为: -999
列 'measured_albedo' 的所有值相同，值为: -999
列 'surface_temperature(meas)' 的所有值相同，值为: -999
列 '24h_wind_drift' 的所有值相同，值为: 0.0
列 'virtual_lysimeter_under_the_soil' 的所有值相同，值为: -999
列 'temperature@0.25m' 的所有值相同，值为: -999
列 'temperature@0.5m' 的所有值相同，值为: -999
列 'temperature@1m' 的所有值相同，值为: -999
列 'temperature@-0.25m' 的所有值相同，值为: -999
列 'temperature@-0.1m' 的所有值相同，值为: -999
列 'profile_type' 的所有值相同，值为: -1.0
列 'z_S5' 的所有值相同，值为: 0.0


In [2]:
import pandas as pd

# 定义文件路径
file_path = '/home/develop/Station/data/csv/station.csv'

# 读取 CSV 文件
data = pd.read_csv(file_path, header=0, sep=',', low_memory=False)

# 定义要删除的列名列表
columns_to_drop = [
    "ground_heat", "measured_albedo", "surface_temperature(meas)", 
    "24h_wind_drift", "erosion_mass_loss", "virtual_lysimeter_under_the_soil", 
    "temperature@0.25m", "temperature@0.5m", "temperature@1m", 
    "temperature@-0.25m", "temperature@-0.1m", "profile_type", "z_S5"
]

# 删除指定列
data.drop(columns=columns_to_drop, inplace=True)

# 保存修改后的数据回到 CSV 文件
data.to_csv(file_path, index=False)

print("已删除指定的列:", columns_to_drop)


已删除指定的列: ['ground_heat', 'measured_albedo', 'surface_temperature(meas)', '24h_wind_drift', 'erosion_mass_loss', 'virtual_lysimeter_under_the_soil', 'temperature@0.25m', 'temperature@0.5m', 'temperature@1m', 'temperature@-0.25m', 'temperature@-0.1m', 'profile_type', 'z_S5']


### 多少行包含缺失值

In [1]:
import pandas as pd
file_path = '/home/develop/Station/data/csv/station.csv'
data = pd.read_csv(file_path, header=0, sep=',', na_values=-999, low_memory=False)
rows_with_missing_values = data.isnull().any(axis=1).sum()
print(f"包含缺失值的行数: {rows_with_missing_values}")

包含缺失值的行数: 0


### 哪些列包含缺失值

In [4]:
import pandas as pd
import numpy as np

# 定义文件路径
file_path = '/home/develop/Station/data/csv/station.csv'

data = pd.read_csv(file_path, header=0, sep=',', na_values=-999, low_memory=False)

columns_with_missing_values = data.columns[data.isnull().any()]

print(f"包含缺失值的列数: {len(columns_with_missing_values)}")
print("包含缺失值的列名:")
for col in columns_with_missing_values:
    print(col)

包含缺失值的列数: 2
包含缺失值的列名:
ground_heat_at_soil_interface
stability_index_5


### 删除包含缺失值的列

In [5]:
import pandas as pd

# 定义文件路径
file_path = '/home/develop/Station/data/csv/station.csv'

# 读取 CSV 文件
data = pd.read_csv(file_path, header=0, sep=',', low_memory=False)

# 定义要删除的列名列表
columns_to_drop = [
    "ground_heat_at_soil_interface",  "stability_index_5"
]

# 删除指定列
data.drop(columns=columns_to_drop, inplace=True)

# 保存修改后的数据回到 CSV 文件
data.to_csv(file_path, index=False)

print("已删除指定的列:", columns_to_drop)


已删除指定的列: ['ground_heat_at_soil_interface', 'stability_index_5']


### 输出文件形状

In [6]:
import pandas as pd
import numpy as np

file_path = '/home/develop/Station/data/csv/station.csv'
data = pd.read_csv(file_path, header=0, sep=',',low_memory=False)
x,y = data.shape
print(f'行数:{x},列数:{y}')

行数:1901,列数:43


### 对数据进行归一化

In [16]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 读取原始数据
file_path = '/home/develop/Station/data/csv/station.csv'
data = pd.read_csv(file_path)

# 初始化 MinMaxScaler
scaler = MinMaxScaler()

# 对数据进行归一化，忽略非数值列（如果有的话）
# 创建一个副本，避免修改原始数据
data_normalized = data.copy()

# 假设第一列为时间戳，我们从第二列开始进行归一化
# 如果有其他非数值列，需要进一步调整以下代码来排除这些列
if not data_normalized.select_dtypes(include=['number']).empty:
    numeric_columns = data_normalized.select_dtypes(include=['number']).columns
    data_normalized[numeric_columns] = scaler.fit_transform(data_normalized[numeric_columns])

# 保存归一化后的数据
output_path = '/home/develop/Station/data/csv/station_normalized.csv'
data_normalized.to_csv(output_path, index=False)

print(f"归一化后的数据已保存到 {output_path}")

归一化后的数据已保存到 /home/develop/Station/data/csv/station_normalized.csv


In [17]:
import pandas as pd
import numpy as np

file_path = '/home/develop/Station/data/csv/station_normalized.csv'
data = pd.read_csv(file_path, header=0, sep=',',low_memory=False)
x,y = data.shape
print(f'行数:{x},列数:{y}')

行数:1901,列数:43


### 将雪崩前后15h的数据添加到测试集，并从原文件中删除

In [18]:
import pandas as pd

# 文件路径
file_path = '/home/develop/Station/data/csv/station_normalized.csv'
test_file_path = '/home/develop/Station/data/csv/test_station.csv'
df = pd.read_csv(file_path)
subset_df = df.iloc[1394:1425]
subset_df.to_csv(test_file_path, index=False)
df = df.drop(df.index[1394:1425])
df.to_csv(file_path, index=False)


In [19]:
import pandas as pd
file_path1 = '/home/develop/Station/data/csv/station_normalized.csv'
file_path2 = '/home/develop/Station/data/csv/test_station.csv'
data1 = pd.read_csv(file_path1, header=0, sep=',',low_memory=False)
data2 = pd.read_csv(file_path2, header=0, sep=',',low_memory=False)
x1,y1 = data1.shape
x2,y2 = data2.shape
print(f'源文件行数:{x1},列数:{y1}')
print(f'测试集行数:{x2},列数:{y2}')

源文件行数:1870,列数:43
测试集行数:31,列数:43


### 从源文件中随机选取160条数据添加到测试集兵打乱顺序

In [20]:
import pandas as pd
import numpy as np

# 读取源文件
source_file_path = '/home/develop/Station/data/csv/station_normalized.csv'
data = pd.read_csv(source_file_path)

# 从源文件中随机选取160条数据
random_sample = data.sample(n=160, random_state=42)

# 读取目标文件，如果文件不存在则创建一个空的 DataFrame
target_file_path = '/home/develop/Station/data/csv/test_station.csv'
try:
    target_data = pd.read_csv(target_file_path)
except FileNotFoundError:
    target_data = pd.DataFrame()

# 将选取的数据添加到目标文件
target_data = pd.concat([target_data, random_sample], ignore_index=True)

# 保存更新后的目标文件
target_data.to_csv(target_file_path, index=False)

# 删除源文件中选中的160条数据
data = data.drop(random_sample.index)

# 打散源文件顺序
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# 保存更新后的源文件
data.to_csv(source_file_path, index=False)

print(f"Successfully added 160 rows to {target_file_path} and updated {source_file_path}.")


Successfully added 160 rows to /home/develop/Station/data/csv/test_station.csv and updated /home/develop/Station/data/csv/station_normalized.csv.


In [21]:
import pandas as pd
file_path1 = '/home/develop/Station/data/csv/station_normalized.csv'
file_path2 = '/home/develop/Station/data/csv/test_station.csv'
data1 = pd.read_csv(file_path1, header=0, sep=',',low_memory=False)
data2 = pd.read_csv(file_path2, header=0, sep=',',low_memory=False)
x1,y1 = data1.shape
x2,y2 = data2.shape
print(f'源文件行数:{x1},列数:{y1}')
print(f'测试集行数:{x2},列数:{y2}')

源文件行数:1710,列数:43
测试集行数:191,列数:43


### 使用训练集训练训练GATv2模型

In [23]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse
from sklearn.model_selection import ParameterGrid

# 读取数据
file_path = '/home/develop/Station/data/csv/station_normalized.csv'
data = pd.read_csv(file_path)
data = data.drop(columns=data.columns[0])
features = torch.tensor(data.values, dtype=torch.float32)  

# 转置特征矩阵，适配 GAT 输入 (节点数, 特征维度)
features = features.T  # 

# 创建边索引（完全图假设，每个特征和其他特征都有边）
num_features = features.size(0)
adj_matrix = torch.ones((num_features, num_features)) - torch.eye(num_features)  # 完全图
edge_index = dense_to_sparse(adj_matrix)[0]

# 构造图数据
graph_data = Data(x=features, edge_index=edge_index)

# GATv2 模型定义
class GATv2Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

# 网格搜索的超参数组合
param_grid = {
    'hidden_channels': [8, 16, 32],
    'heads': [1, 2, 4],
    'learning_rate': [0.001, 0.005],
    'weight_decay': [0.0, 1e-4]
}
grid = ParameterGrid(param_grid)

# 早停策略
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def step(self, loss):
        if loss < self.best_loss:
            self.best_loss = loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 日志保存路径
log_dir = "/home/develop/Station/Result"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, "GATv2_Train.log")

# 训练过程
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

with open(log_file, "w") as log:
    for params in grid:
        # 打印当前超参数组合
        log.write(f"Training with params: {params}\n")
        print(f"Training with params: {params}")
        
        # 模型实例化
        model = GATv2Net(
            in_channels=features.size(1),
            hidden_channels=params['hidden_channels'],
            out_channels=features.size(1),
            heads=params['heads']
        ).to(device)

        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
        loss_fn = nn.MSELoss()

        graph_data = graph_data.to(device)
        model.train()
        early_stopping = EarlyStopping(patience=10)

        for epoch in range(100):  # 最大训练轮数
            optimizer.zero_grad()
            out = model(graph_data.x, graph_data.edge_index)  # 前向传播
            loss = loss_fn(out, graph_data.x)  # 重构损失
            loss.backward()  # 反向传播
            optimizer.step()  # 参数更新

            early_stopping.step(loss.item())
            if early_stopping.early_stop:
                break

        # 记录结果
        final_loss = early_stopping.best_loss
        results.append({
            'params': params,
            'final_loss': final_loss
        })

        log.write(f"Final Loss for params {params}: {final_loss:.6f}\n")
        log.write("-" * 50 + "\n")
        print(f"Final Loss for params {params}: {final_loss:.6f}")

# 打印最佳超参数组合
best_result = min(results, key=lambda x: x['final_loss'])
with open(log_file, "a") as log:
    log.write("\nBest Hyperparameters:\n")
    log.write(str(best_result['params']) + "\n")
    log.write(f"Best Loss: {best_result['final_loss']:.6f}\n")

print("\nBest Hyperparameters:", best_result['params'])
print("Best Loss:", best_result['final_loss'])


Training with params: {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.001, 'weight_decay': 0.0}
Final Loss for params {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.001, 'weight_decay': 0.0}: 0.054801
Training with params: {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.001, 'weight_decay': 0.0001}
Final Loss for params {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.001, 'weight_decay': 0.0001}: 0.050240
Training with params: {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.005, 'weight_decay': 0.0}
Final Loss for params {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.005, 'weight_decay': 0.0}: 0.124047
Training with params: {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.005, 'weight_decay': 0.0001}
Final Loss for params {'heads': 1, 'hidden_channels': 8, 'learning_rate': 0.005, 'weight_decay': 0.0001}: 0.124141
Training with params: {'heads': 1, 'hidden_channels': 16, 'learning_rate': 0.001, 'weight_decay': 0.0}
Final Loss for params {'heads

### Best Hyperparameters:

{'heads': 2, 'hidden_channels': 8, 'learning_rate': 0.005, 'weight_decay': 0.0001}

**Best Loss: 0.04161439463496208**

### 根据确定超参数训练并保存模型

In [24]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse
import pandas as pd


# 超参数
hidden_channels = 8  # 隐藏层维度
heads = 2  # 多头注意力
learning_rate = 0.005  # 学习率
weight_decay = 0.0001  # 权重衰减
epochs = 100  # 最大训练轮数
patience = 10  # 早停容忍次数
save_path = "/home/develop/Station/Model/GATv2_trained.pth"  # 保存路径
log_file = "/home/develop/Station/Result/GATv2_Train.log"  # 日志路径

# 数据加载与预处理
file_path = '/home/develop/Station/data/csv/station_normalized.csv'
data = pd.read_csv(file_path)
data = data.drop(columns=data.columns[0])
features = torch.tensor(data.values, dtype=torch.float32) 

# 转置特征矩阵，适配 GAT 输入 (节点数, 特征维度)
features = features.T

# 创建边索引（完全图假设，每个特征和其他特征都有边）
num_features = features.size(0)
adj_matrix = torch.ones((num_features, num_features)) - torch.eye(num_features)  # 完全图
edge_index = dense_to_sparse(adj_matrix)[0]

# 构造图数据
graph_data = Data(x=features, edge_index=edge_index)

# GATv2 模型定义
class GATv2Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

# 早停机制
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def step(self, loss):
        if loss < self.best_loss:
            self.best_loss = loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 确保日志目录存在
os.makedirs(os.path.dirname(log_file), exist_ok=True)

# 初始化模型和优化器
device = torch.device('cpu')  # 强制使用 CPU
model = GATv2Net(
    in_channels=features.size(1),
    hidden_channels=hidden_channels,
    out_channels=features.size(1),  # 输出维度等于输入维度（重构任务）
    heads=heads
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fn = nn.MSELoss()

# 确保 graph_data 的所有元素移动到设备
graph_data = graph_data.to(device)

# 模型训练
model.train()
early_stopping = EarlyStopping(patience=patience)

with open(log_file, "w") as log:
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(graph_data.x, graph_data.edge_index)  # 前向传播
        loss = loss_fn(out, graph_data.x)  # 重构损失
        loss.backward()  # 反向传播
        optimizer.step()  # 参数更新

        # 记录日志
        log.write(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.6f}\n")
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.6f}")

        # 检查早停
        early_stopping.step(loss.item())
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch}")
            log.write(f"Early stopping triggered at epoch {epoch}\n")
            break

# 保存训练好的模型
torch.save(model, save_path)
print(f"Model saved to {save_path}")
with open(log_file, "a") as log:
    log.write(f"Model saved to {save_path}\n")


Epoch 0/100, Loss: 0.246137
Epoch 1/100, Loss: 0.242335
Epoch 2/100, Loss: 0.221502
Epoch 3/100, Loss: 0.209130
Epoch 4/100, Loss: 0.198045
Epoch 5/100, Loss: 0.184760
Epoch 6/100, Loss: 0.168721
Epoch 7/100, Loss: 0.156590
Epoch 8/100, Loss: 0.147974
Epoch 9/100, Loss: 0.140580
Epoch 10/100, Loss: 0.137008
Epoch 11/100, Loss: 0.131616
Epoch 12/100, Loss: 0.129241
Epoch 13/100, Loss: 0.127896
Epoch 14/100, Loss: 0.127150
Epoch 15/100, Loss: 0.126830
Epoch 16/100, Loss: 0.126769
Epoch 17/100, Loss: 0.126708
Epoch 18/100, Loss: 0.126368
Epoch 19/100, Loss: 0.126004
Epoch 20/100, Loss: 0.125727
Epoch 21/100, Loss: 0.125502
Epoch 22/100, Loss: 0.125308
Epoch 23/100, Loss: 0.125096
Epoch 24/100, Loss: 0.124886
Epoch 25/100, Loss: 0.124762
Epoch 26/100, Loss: 0.124687
Epoch 27/100, Loss: 0.124630
Epoch 28/100, Loss: 0.124571
Epoch 29/100, Loss: 0.124538
Epoch 30/100, Loss: 0.124498
Epoch 31/100, Loss: 0.124465
Epoch 32/100, Loss: 0.124443
Epoch 33/100, Loss: 0.124427
Epoch 34/100, Loss: 0.12

### 使用GATv2进行推理

In [25]:
import torch
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch.nn as nn
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse
from sklearn.model_selection import ParameterGrid

class GATv2Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x
file_path = '/home/develop/Station/data/csv/station_normalized.csv'
data_new = pd.read_csv(file_path)
data_new = data_new.drop(columns=data_new.columns[0])

features_new = torch.tensor(data_new.values, dtype=torch.float32).T 
num_features_new = features_new.size(0)

# 3. 创建边索引（完全图假设，每个特征和其他特征都有边）
adj_matrix_new = torch.ones((num_features_new, num_features_new)) - torch.eye(num_features_new)  # 完全图
edge_index_new = dense_to_sparse(adj_matrix_new)[0]

# 4. 创建图数据对象
device = torch.device('cpu')  # 使用 CPU
graph_data_new = Data(x=features_new, edge_index=edge_index_new)

# 5. 加载训练好的 GATv2 模型
model = torch.load("/home/develop/Station/Model/GATv2_trained.pth")
model.eval()  # 设置为评估模式

# 6. 确保数据移到正确的设备
graph_data_new = graph_data_new.to(device)

# 7. 推理
with torch.no_grad():  # 禁用梯度计算
    output_new = model(graph_data_new.x, graph_data_new.edge_index)  # 获得模型输出

# 8. 打印输出
print("GATv2 模型的输出：")
print(output_new)  # 打印推理结果

# 9. 输出数据的维度，确认是否匹配预期
print(f"模型输出的维度: {output_new.shape}")


GATv2 模型的输出：
tensor([[0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109],
        [0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109],
        [0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109],
        ...,
        [0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109],
        [0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109],
        [0.3034, 0.3667, 0.3797,  ..., 0.1778, 0.3351, 0.3109]])
模型输出的维度: torch.Size([42, 1710])


/tmp/ipykernel_649945/658681173.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/home/develop/Station/Model/GATv2_trained.pth")


In [26]:
output_new_transposed = output_new.T  # 转置后变为 [1186, 42]
output_new_transposed_np = output_new_transposed.detach().numpy()  # 转换为 NumPy 数组

print(output_new_transposed.shape)
print(type(output_new_transposed))
output_new_transposed_df = pd.DataFrame(output_new_transposed.numpy())

print(output_new_transposed_df.shape)
print(type(output_new_transposed_df))
print(output_new_transposed_df)

torch.Size([1710, 42])
<class 'torch.Tensor'>
(1710, 42)
<class 'pandas.core.frame.DataFrame'>
            0         1         2         3         4         5         6   \
0     0.303360  0.303360  0.303360  0.303360  0.303360  0.303360  0.303360   
1     0.366682  0.366682  0.366682  0.366682  0.366682  0.366682  0.366682   
2     0.379706  0.379706  0.379706  0.379706  0.379706  0.379706  0.379706   
3     0.283430  0.283430  0.283430  0.283430  0.283430  0.283430  0.283430   
4     0.439225  0.439225  0.439225  0.439225  0.439225  0.439225  0.439225   
...        ...       ...       ...       ...       ...       ...       ...   
1705  0.393685  0.393685  0.393685  0.393685  0.393685  0.393685  0.393685   
1706  0.359825  0.359825  0.359825  0.359825  0.359825  0.359825  0.359825   
1707  0.177849  0.177849  0.177849  0.177849  0.177849  0.177849  0.177849   
1708  0.335140  0.335140  0.335140  0.335140  0.335140  0.335140  0.335140   
1709  0.310869  0.310869  0.310869  0.310869  0

In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model
from datetime import datetime

# 假设output_new_transposed_df是GATv2模型的输出DataFrame
# output_new_transposed_df = your_data_here

# 1. 数据预处理
X = output_new_transposed_df.values  # 转为numpy数组

# 划分训练集和验证集，按8:2划分
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# 定义采样层
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# 定义编码器部分为独立的模型
class Encoder(layers.Layer):
    def __init__(self, latent_dim, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.dense_1 = layers.Dense(64, activation="relu")
        self.dense_2 = layers.Dense(32, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling_layer = layers.Lambda(sampling)

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling_layer([z_mean, z_log_var])
        return z_mean, z_log_var, z

# 定义解码器部分为独立的模型
class Decoder(layers.Layer):
    def __init__(self, original_dim, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.dense_1 = layers.Dense(32, activation="relu")
        self.dense_2 = layers.Dense(64, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.dense_output(x)

# 定义VAE模型
class VAE(Model):
    def __init__(self, original_dim, latent_dim, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim)
        self.decoder = Decoder(original_dim=original_dim)
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss_fn = tf.keras.losses.MeanSquaredError()
            reconstruction_loss = tf.reduce_mean(reconstruction_loss_fn(data, reconstruction))
            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1
            )
            total_loss = reconstruction_loss + 0.1 * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss_fn = tf.keras.losses.MeanSquaredError()
        reconstruction_loss = tf.reduce_mean(reconstruction_loss_fn(data, reconstruction))
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1
        )
        total_loss = reconstruction_loss + 0.1 * kl_loss
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        return self.decoder(z)

# 设置超参数网格
latent_dims = [5, 10, 15, 20]  # 潜在空间维度
batch_sizes = [16, 32, 64]
epochs_list = [20, 30, 50]
log_file = "/home/develop/Station/Result/GATv2VAE.log"
save_dir = "/home/develop/Station/Result/pic/"
os.makedirs(save_dir, exist_ok=True)

# 网格搜索训练过程
with open(log_file, "w") as log:
    log.write("Training Log\n")
    log.write("Parameters: latent_dim, batch_size, epochs\n")
    log.write("Results: reconstruction_error_threshold, anomalies_detected, training_time\n")
    log.write("-" * 80 + "\n")

    for latent_dim in latent_dims:
        for batch_size in batch_sizes:
            for epochs in epochs_list:
                input_dim = X_train.shape[1]

                # 构建VAE模型
                vae = VAE(original_dim=input_dim, latent_dim=latent_dim)
                vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

                # 训练模型
                start_time = datetime.now()
                history = vae.fit(
                    X_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test,),  # 只传递验证数据，不包括目标
                    verbose=0
                )
                training_time = datetime.now() - start_time

                # 绘制训练损失
                plt.figure(figsize=(10, 6))
                
                # 检查是否存在验证损失键
                if 'val_loss' in history.history:
                    plt.plot(history.history["loss"], label="Train Loss")
                    plt.plot(history.history["val_loss"], label="Validation Loss")
                else:
                    plt.plot(history.history["loss"], label="Loss")
                
                plt.xlabel("Epoch")
                plt.ylabel("Loss")
                plt.legend()
                plt.title(f"Loss (latent_dim={latent_dim}, batch_size={batch_size}, epochs={epochs})")
                plt.savefig(f"{save_dir}loss_latent{latent_dim}_batch{batch_size}_epochs{epochs}.png")
                plt.close()

                # 异常检测
                X_pred = vae.predict(X_test, verbose=0)
                reconstruction_error = np.mean(np.square(X_test - X_pred), axis=1)
                threshold = np.percentile(reconstruction_error, 95)
                anomalies = reconstruction_error > threshold

                # 绘制重构误差分布
                plt.figure()
                plt.hist(reconstruction_error, bins=50)
                plt.xlabel("Reconstruction Error")
                plt.ylabel("Number of Samples")
                plt.title(f"Error Dist. (latent_dim={latent_dim}, batch_size={batch_size}, epochs={epochs})")
                plt.savefig(f"{save_dir}error_dist_latent{latent_dim}_batch{batch_size}_epochs{epochs}.png")
                plt.close()

                # 记录日志
                log.write(f"latent_dim={latent_dim}, batch_size={batch_size}, epochs={epochs}\n")
                log.write(f"reconstruction_error_threshold={threshold:.4f}, anomalies_detected={np.sum(anomalies)}, training_time={training_time}\n")
                log.write("-" * 80 + "\n")
                print(f"Params: latent_dim={latent_dim}, batch_size={batch_size}, epochs={epochs}")
                print(f"Reconstruction Error Threshold: {threshold:.4f}")
                print(f"Anomalies detected: {np.sum(anomalies)}")
                print(f"Training Time: {training_time}\n")

2024-12-19 16:32:20.457867: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 16:32:20.502608: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 16:32:20.503578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 16:32:21.276586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Params: latent_dim=5, batch_size=16, epochs=20
Reconstruction Error Threshold: 0.0153
Anomalies detected: 18
Training Time: 0:00:03.595765

Params: latent_dim=5, batch_size=16, epochs=30
Reconstruction Error Threshold: 0.0149
Anomalies detected: 18
Training Time: 0:00:04.653367

Params: latent_dim=5, batch_size=16, epochs=50
Reconstruction Error Threshold: 0.0143
Anomalies detected: 18
Training Time: 0:00:06.900695

Params: latent_dim=5, batch_size=32, epochs=20
Reconstruction Error Threshold: 0.0135
Anomalies detected: 18
Training Time: 0:00:02.578437

Params: latent_dim=5, batch_size=32, epochs=30
Reconstruction Error Threshold: 0.0146
Anomalies detected: 18
Training Time: 0:00:03.312582

Params: latent_dim=5, batch_size=32, epochs=50
Reconstruction Error Threshold: 0.0145
Anomalies detected: 18
Training Time: 0:00:04.632964

Params: latent_dim=5, batch_size=64, epochs=20
Reconstruction Error Threshold: 0.0190
Anomalies detected: 18
Training Time: 0:00:02.159168

Params: latent_dim=5

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from collections import Counter

# GATv2 模型定义
class GATv2Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

def prepare_data(file_path):
    data = pd.read_csv(file_path)
    
    # 移除 timestamp 列，如果存在的话
    if 'timestamp' in data.columns:
        features = data.drop(columns=['timestamp']).values
    else:
        features = data.values
    
    features = torch.tensor(features, dtype=torch.float32).T 
    num_features = features.size(0)
    
    adj_matrix = torch.ones((num_features, num_features)) - torch.eye(num_features)  # 完全图
    edge_index = dense_to_sparse(adj_matrix)[0]
    
    return Data(x=features, edge_index=edge_index)

def inference(model, file_path):
    model.eval()  # 设置为评估模式
    graph_data_new = prepare_data(file_path).to(device)

    with torch.no_grad():
        output_new = model(graph_data_new.x, graph_data_new.edge_index)  # 获得模型输出

    return output_new.T.numpy()  # 返回转置后的numpy数组

# VAE相关部分
def build_vae(input_dim, latent_dim):
    class Encoder(layers.Layer):
        def __init__(self, latent_dim, **kwargs):
            super(Encoder, self).__init__(**kwargs)
            self.dense_1 = layers.Dense(64, activation="relu")
            self.dense_2 = layers.Dense(32, activation="relu")
            self.dense_mean = layers.Dense(latent_dim)
            self.dense_log_var = layers.Dense(latent_dim)

        def call(self, inputs):
            x = self.dense_1(inputs)
            x = self.dense_2(x)
            z_mean = self.dense_mean(x)
            z_log_var = self.dense_log_var(x)
            return z_mean, z_log_var

    class Decoder(layers.Layer):
        def __init__(self, original_dim, **kwargs):
            super(Decoder, self).__init__(**kwargs)
            self.dense_1 = layers.Dense(32, activation="relu")
            self.dense_2 = layers.Dense(64, activation="relu")
            self.dense_output = layers.Dense(original_dim, activation="sigmoid")

        def call(self, inputs):
            x = self.dense_1(inputs)
            x = self.dense_2(x)
            return self.dense_output(x)

    class VAE(Model):
        def __init__(self, original_dim, latent_dim, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.original_dim = original_dim
            self.encoder = Encoder(latent_dim=latent_dim)
            self.decoder = Decoder(original_dim=original_dim)

        def call(self, inputs):
            z_mean, z_log_var = self.encoder(inputs)
            epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
            z = z_mean + tf.exp(0.5 * z_log_var) * epsilon
            return self.decoder(z)

    vae = VAE(original_dim=input_dim, latent_dim=latent_dim)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
    return vae

def run_inference_multiple_times(model, vae, data_file_path, iterations=75):
    anomaly_counts = Counter()
    data = pd.read_csv(data_file_path)
    
    for _ in range(iterations):
        output_new_transposed = inference(model, data_file_path)
        output_new_transposed_df = pd.DataFrame(output_new_transposed)
        
        X_pred = vae.predict(output_new_transposed_df.values, batch_size=16, verbose=0)
        reconstruction_error = np.mean(np.square(output_new_transposed_df.values - X_pred), axis=1)
        
        threshold = np.percentile(reconstruction_error, 95)
        anomalies = reconstruction_error > threshold
        
        anomalous_indices = np.where(anomalies)[0]
        for index in anomalous_indices:
            anomaly_counts[index] += 1
    
    # 获取被判定为异常次数最多的前15个数据点
    top_anomalies = anomaly_counts.most_common(15)
    top_anomalous_indices = [index for index, count in top_anomalies]

    # 确认 'timestamp' 列存在并获取其值
    if 'timestamp' not in data.columns:
        print("No timestamp column found.")
        timestamp_values = ['N/A'] * len(top_anomalous_indices)
    else:
        timestamp_values = data.loc[top_anomalous_indices, 'timestamp'].values
    
    return top_anomalous_indices, timestamp_values

# 超参数
hidden_channels = 8  # 隐藏层维度
heads = 2  # 多头注意力

# 数据路径
data_file_path = '/home/develop/Station/data/csv/test_station.csv'

# 设备配置
device = torch.device('cpu')  # 强制使用 CPU

# 初始化模型（这里假设模型的输入和输出维度与训练时相同）
graph_data = prepare_data(data_file_path)
in_channels = graph_data.x.size(1)
out_channels = graph_data.x.size(1)  # 输出维度等于输入维度（重构任务）

model = GATv2Net(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,  # 输出维度等于输入维度（重构任务）
    heads=heads
).to(device)

# 构建VAE模型
latent_dim = 20
input_dim = graph_data.x.size(0)

vae = build_vae(input_dim, latent_dim)

# 执行多次推理并获取结果
top_anomalous_indices, timestamp_values = run_inference_multiple_times(model, vae, data_file_path)

# 输出结果
print("Top 15 anomalous indices and their 'timestamp' values:")
for index, timestamp in zip(top_anomalous_indices, timestamp_values):
    print(f"Index: {index}, timestamp: {timestamp}")

Top 15 anomalous indices and their 'timestamp' values:
Index: 0, timestamp: 2024-02-04T02:00:00
Index: 16, timestamp: 2024-02-04T18:00:00
Index: 17, timestamp: 2024-02-04T19:00:00
Index: 61, timestamp: 2024-01-08T20:00:00
Index: 100, timestamp: 2024-01-30T17:00:00
Index: 134, timestamp: 2024-02-20T06:00:00
Index: 156, timestamp: 2023-12-20T19:00:00
Index: 172, timestamp: 2023-12-20T01:00:00
Index: 37, timestamp: 2023-12-18T07:00:00
Index: 68, timestamp: 2024-02-24T19:00:00
Index: 54, timestamp: 2024-01-31T05:00:00
Index: 102, timestamp: 2024-01-25T08:00:00


In [29]:
import pandas as pd

# 读取目标文件
file_path = '/home/develop/Station/data/csv/test_station.csv'
data = pd.read_csv(file_path)

# 打散数据顺序
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# 保存打散后的数据
shuffled_data.to_csv(file_path, index=False)

print("The data has been shuffled and saved.")


The data has been shuffled and saved.


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from collections import Counter

# GATv2 模型定义
class GATv2Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

def prepare_data(file_path):
    data = pd.read_csv(file_path)
    
    # 移除 timestamp 列，如果存在的话
    if 'timestamp' in data.columns:
        features = data.drop(columns=['timestamp']).values
    else:
        features = data.values
    
    features = torch.tensor(features, dtype=torch.float32).T 
    num_features = features.size(0)
    
    adj_matrix = torch.ones((num_features, num_features)) - torch.eye(num_features)  # 完全图
    edge_index = dense_to_sparse(adj_matrix)[0]
    
    return Data(x=features, edge_index=edge_index)

def inference(model, file_path):
    model.eval()  # 设置为评估模式
    graph_data_new = prepare_data(file_path).to(device)

    with torch.no_grad():
        output_new = model(graph_data_new.x, graph_data_new.edge_index)  # 获得模型输出

    return output_new.T.numpy()  # 返回转置后的numpy数组

# VAE相关部分
def build_vae(input_dim, latent_dim):
    class Encoder(layers.Layer):
        def __init__(self, latent_dim, **kwargs):
            super(Encoder, self).__init__(**kwargs)
            self.dense_1 = layers.Dense(64, activation="relu")
            self.dense_2 = layers.Dense(32, activation="relu")
            self.dense_mean = layers.Dense(latent_dim)
            self.dense_log_var = layers.Dense(latent_dim)

        def call(self, inputs):
            x = self.dense_1(inputs)
            x = self.dense_2(x)
            z_mean = self.dense_mean(x)
            z_log_var = self.dense_log_var(x)
            return z_mean, z_log_var

    class Decoder(layers.Layer):
        def __init__(self, original_dim, **kwargs):
            super(Decoder, self).__init__(**kwargs)
            self.dense_1 = layers.Dense(32, activation="relu")
            self.dense_2 = layers.Dense(64, activation="relu")
            self.dense_output = layers.Dense(original_dim, activation="sigmoid")

        def call(self, inputs):
            x = self.dense_1(inputs)
            x = self.dense_2(x)
            return self.dense_output(x)

    class VAE(Model):
        def __init__(self, original_dim, latent_dim, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.original_dim = original_dim
            self.encoder = Encoder(latent_dim=latent_dim)
            self.decoder = Decoder(original_dim=original_dim)

        def call(self, inputs):
            z_mean, z_log_var = self.encoder(inputs)
            epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
            z = z_mean + tf.exp(0.5 * z_log_var) * epsilon
            return self.decoder(z)

    vae = VAE(original_dim=input_dim, latent_dim=latent_dim)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
    return vae

def run_inference_multiple_times(model, vae, data_file_path, iterations=150):
    anomaly_counts = Counter()
    data = pd.read_csv(data_file_path)
    
    for _ in range(iterations):
        output_new_transposed = inference(model, data_file_path)
        output_new_transposed_df = pd.DataFrame(output_new_transposed)
        
        X_pred = vae.predict(output_new_transposed_df.values, batch_size=16, verbose=0)
        reconstruction_error = np.mean(np.square(output_new_transposed_df.values - X_pred), axis=1)
        
        threshold = np.percentile(reconstruction_error, 95)
        anomalies = reconstruction_error > threshold
        
        anomalous_indices = np.where(anomalies)[0]
        for index in anomalous_indices:
            anomaly_counts[index] += 1
    
    # 获取被判定为异常次数最多的前15个数据点
    top_anomalies = anomaly_counts.most_common(15)
    top_anomalous_indices = [index for index, count in top_anomalies]

    # 确认 'timestamp' 列存在并获取其值
    if 'timestamp' not in data.columns:
        print("No timestamp column found.")
        timestamp_values = ['N/A'] * len(top_anomalous_indices)
    else:
        timestamp_values = data.loc[top_anomalous_indices, 'timestamp'].values
    
    return top_anomalous_indices, timestamp_values

# 超参数
hidden_channels = 8  # 隐藏层维度
heads = 2  # 多头注意力

# 数据路径
data_file_path = '/home/develop/Station/data/csv/test_station.csv'

# 设备配置
device = torch.device('cpu')  # 强制使用 CPU

# 初始化模型（这里假设模型的输入和输出维度与训练时相同）
graph_data = prepare_data(data_file_path)
in_channels = graph_data.x.size(1)
out_channels = graph_data.x.size(1)  # 输出维度等于输入维度（重构任务）

model = GATv2Net(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,  # 输出维度等于输入维度（重构任务）
    heads=heads
).to(device)

# 构建VAE模型
latent_dim = 20
input_dim = graph_data.x.size(0)

vae = build_vae(input_dim, latent_dim)

# 执行多次推理并获取结果
top_anomalous_indices, timestamp_values = run_inference_multiple_times(model, vae, data_file_path)

# 输出结果
print("Top 15 anomalous indices and their 'timestamp' values:")
for index, timestamp in zip(top_anomalous_indices, timestamp_values):
    print(f"Index: {index}, timestamp: {timestamp}")

Top 15 anomalous indices and their 'timestamp' values:
Index: 12, timestamp: 2024-02-04T20:00:00
Index: 20, timestamp: 2024-02-05T08:00:00
Index: 49, timestamp: 2024-01-26T02:00:00
Index: 51, timestamp: 2024-01-31T14:00:00
Index: 76, timestamp: 2024-02-04T13:00:00
Index: 93, timestamp: 2023-12-08T23:00:00
Index: 111, timestamp: 2023-12-25T17:00:00
Index: 178, timestamp: 2024-02-01T16:00:00
Index: 30, timestamp: 2024-01-03T07:00:00
Index: 158, timestamp: 2024-01-03T13:00:00
Index: 78, timestamp: 2024-02-17T02:00:00
Index: 86, timestamp: 2024-01-16T07:00:00
Index: 104, timestamp: 2024-02-05T03:00:00
